In [92]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
import sklearn as sk

Consider four possible models for predicting house prices:

Using only the size and number of rooms.
Using size, number of rooms, and building type.
Using size and building type, and their interaction.
Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and also building type.
Set up a pipeline for each of these four models.

Then, get predictions on the test set for each of your pipelines, and compute the root mean squared error. Which model performed best?

Note: You should only use the function train_test_split() one time in your code; that is, we should be predicting on the same test set for all three models.

## Part 1

In [93]:

housing = pd.read_csv("/Users/williamkapner/Documents/GSB_544/Data/AmesHousing.csv")
housing.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [94]:
X = housing.drop("SalePrice", axis = 1)
y = housing["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [95]:
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(
  [
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
  ],
  remainder = "drop"
)


lr_pipeline = Pipeline(
  [("preprocessing", ct),
  ("linear_regression", LinearRegression())]
)



lr_fitted = lr_pipeline.fit(X_train, y_train)



# Predictions for train and test sets
y_train_pred = lr_fitted.predict(X_train)
y_test_pred = lr_fitted.predict(X_test)


# Calculate MSE for training and testing data
r1 = r2_score(y_test, y_test_pred)
intercept1 = lr_fitted.named_steps['linear_regression'].intercept_
coefficients1 = lr_fitted.named_steps['linear_regression'].coef_
crossval1 = cross_val_score(lr_pipeline, X_train, y_train, cv=5, scoring="neg_mean_squared_error")
rmse1 = np.sqrt(-crossval1)
mse1 = sk.metrics.mean_squared_error(y_test, y_test_pred)

print("R_Squared:", r1)
print("Cross Validation RMSE:", rmse1.mean())
print("Slope:", coefficients1)
print("Intercept:", intercept1)

R_Squared: 0.5375267745996803
Cross Validation RMSE: 55174.98589881498
Slope: [ 71638.1428861  -21582.11735524]
Intercept: 180368.25079654073


In [96]:
from sklearn.compose import ColumnTransformer

ct1 = ColumnTransformer(
  [  
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"]),
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"])
  ],
  remainder = "drop"
)


lr_pipeline = Pipeline(
  [("preprocessing", ct1),
  ("linear_regression", LinearRegression())]
)


lr_fitted = lr_pipeline.fit(X_train, y_train)


# Predictions for train and test sets
y_train_pred = lr_fitted.predict(X_train)
y_test_pred = lr_fitted.predict(X_test)


# Calculate MSE for training and testing data
r2 = r2_score(y_test, y_test_pred)
intercept2 = lr_fitted.named_steps['linear_regression'].intercept_
coefficients2 = lr_fitted.named_steps['linear_regression'].coef_
crossval2 = cross_val_score(lr_pipeline, X_train, y_train, cv=5, scoring="neg_mean_squared_error")
rmse2 = np.sqrt(-crossval2)
mse2 = sk.metrics.mean_squared_error(y_test, y_test_pred)

print("R_Squared:", r2)
print("Cross Validation RMSE:", rmse2.mean())
print("Slope:", coefficients2)
print("Intercept:", intercept2)

R_Squared: 0.5633220861145007
Cross Validation RMSE: 53356.75472657602
Slope: [ 6.60201811e+04 -1.35416646e+04  1.24597206e+18  1.24597206e+18
  1.24597206e+18  1.24597206e+18  1.24597206e+18]
Intercept: -1.2459720553600315e+18


In [97]:
from sklearn.compose import ColumnTransformer

ct2 = ColumnTransformer(
  [  
    ("standardize", StandardScaler(), ["Gr Liv Area"]),
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"])
  ],
  remainder = "drop"
).set_output(transform="pandas")

X_train_dummified = ct2.fit_transform(X_train)
X_train_dummified

ct_inter = ColumnTransformer(
  [
    ("interaction", PolynomialFeatures(interaction_only = True), ["standardize__Gr Liv Area", "dummify__Bldg Type_1Fam"]),
    ("interaction1", PolynomialFeatures(interaction_only = True), ["standardize__Gr Liv Area", "dummify__Bldg Type_2fmCon"]),
    ("interaction2", PolynomialFeatures(interaction_only = True), ["standardize__Gr Liv Area", "dummify__Bldg Type_Duplex"]),
    ("interaction3", PolynomialFeatures(interaction_only = True), ["standardize__Gr Liv Area", "dummify__Bldg Type_Twnhs"]),
    ("interaction4", PolynomialFeatures(interaction_only = True), ["standardize__Gr Liv Area", "dummify__Bldg Type_TwnhsE"])
  ],
  remainder = "drop"
).set_output(transform = "pandas")

lr_pipeline = Pipeline(
  [("preprocessing", ct2),
   ("interactions", ct_inter),
  ("linear_regression", LinearRegression())]
)


lr_fitted = lr_pipeline.fit(X_train, y_train)


# Predictions for train and test sets
y_test_pred = lr_fitted.predict(X_test)


# Calculate MSE for training and testing data
r3 = r2_score(y_test, y_test_pred)
intercept3 = lr_fitted.named_steps['linear_regression'].intercept_
coefficients3 = lr_fitted.named_steps['linear_regression'].coef_
crossval3 = cross_val_score(lr_pipeline, X_train, y_train, cv=5, scoring="neg_mean_squared_error")
rmse3 = np.sqrt(-crossval3)
mse3 = sk.metrics.mean_squared_error(y_test, y_test_pred)

print("R_Squared:", r3)
print("Cross Validation RMSE:", rmse3.mean())
print("Slope:", coefficients3)
print("Intercept:", intercept3)

R_Squared: 0.5890797792502362
Cross Validation RMSE: 52879.46201230896
Slope: [ 0.00000000e+00 -7.52649751e+17 -1.44688592e+17 -3.80625004e+15
  6.87808878e+15  1.78432192e+17 -1.44688592e+17 -3.80625004e+15
 -7.08403340e+15  1.93006395e+17 -1.44688592e+17 -3.80625004e+15
  0.00000000e+00  1.92508707e+17 -1.44688592e+17 -3.80625004e+15
  7.10542736e-15  1.92508707e+17 -1.44688592e+17 -3.80625004e+15]
Intercept: 1.4489453647987766e+17


In [98]:
from sklearn.compose import ColumnTransformer

ct3 = ColumnTransformer(
  [  
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"]),
    ('polynomial_features', PolynomialFeatures(degree=5, include_bias=False), ["Gr Liv Area", "TotRms AbvGrd"]),
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"])
  ],
  remainder = "drop"
)


lr_pipeline = Pipeline(
  [("preprocessing", ct3),
   
  ("linear_regression", LinearRegression())]
)


lr_fitted = lr_pipeline.fit(X_train, y_train)


# Predictions for train and test sets
y_test_pred = lr_fitted.predict(X_test)


# Calculate MSE for training and testing data
r4 = r2_score(y_test, y_test_pred)
intercept4 = lr_fitted.named_steps['linear_regression'].intercept_
coefficients4 = lr_fitted.named_steps['linear_regression'].coef_
crossval4 = cross_val_score(lr_pipeline, X_train, y_train, cv=5, scoring="neg_mean_squared_error")
rmse4 = np.sqrt(-crossval4)
mse4 = sk.metrics.mean_squared_error(y_test, y_test_pred)
print(mse4)

print("R_Squared:", r4)
print("Cross Validation RMSE:", rmse4.mean())
print("Slope:", coefficients4)
print("Intercept:", intercept4)

3371980394.7024097
R_Squared: 0.5258861594220923
Cross Validation RMSE: 63529.84906106508
Slope: [-1.77530999e-02  7.66181573e-02 -5.71975377e-02 -6.72248502e-03
  2.00386856e-01 -4.05233041e+00 -2.71905433e-02 -1.57383110e-04
  3.41533143e-02 -7.03398539e+00  3.53346589e-02 -5.32979098e-10
  3.27767823e-05 -1.28481968e-02  1.82905671e+00  1.07229452e+00
  3.45791324e-12 -2.52119728e-09 -1.98851130e-06  1.36667988e-03
 -2.52953253e-01  1.04901008e+01 -2.70351590e-03  5.52595254e-05
 -3.96057818e-04  1.17196476e-03  1.87234942e-03]
Intercept: 48570.88673762488


In [99]:
table = {"Model": ["Model 1", "Model 2","Model 3","Model 4"],
         "RMSE": [np.sqrt(mse1),np.sqrt(mse2),np.sqrt(mse3),np.sqrt(mse4)]}

pd.DataFrame(table)

,Model,RMSE
0,Model 1,57351.462994
1,Model 2,55729.072748
2,Model 3,54060.487785
3,Model 4,58068.755753


Here, we can see that Model 3 was the best model. It included interactions which helped reduce the unexplained error in the model and gave us the smallest RMSE.

## Part 2:

Now, we will run the same models using cross validation. The code for the cross validations are included in the code chunks from Part 1.

In [100]:
table1 = {"Model": ["Model 1", "Model 2","Model 3","Model 4"],
         "Cross Validation RMSE": [rmse1.mean(),rmse2.mean(),rmse3.mean(),rmse4.mean()]}

pd.DataFrame(table1)

,Model,Cross Validation RMSE
0,Model 1,55174.985899
1,Model 2,53356.754727
2,Model 3,52879.462012
3,Model 4,63529.849061


Once again, after performing cross validation, Model 3 is still the best model because it has the smallest RMSE after cross validation. This is more powerful than our calculation from Part 1, and confirms that Model 3 with the interactions has the more accurate predictions. This agrees with my earlier conclusion.

## Part 3:

Here, I will create 100 different models with degrees 1 to 10 for the living area and number of rooms variables.

In [101]:
from sklearn.model_selection import GridSearchCV

ct_poly = ColumnTransformer(
  [
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
    ("polynomial_area", PolynomialFeatures(), ["Gr Liv Area"]),
    ("polynomial_rooms", PolynomialFeatures(), ["TotRms AbvGrd"])
    

  ],
  remainder = "drop"
)

lr_pipeline_poly = Pipeline(
  [("preprocessing", ct_poly),
  ("linear_regression", LinearRegression())]
).set_output(transform="pandas")

degrees = {'preprocessing__polynomial_area__degree': np.arange(1, 11),
           'preprocessing__polynomial_rooms__degree': np.arange(1, 11)}

gscv = GridSearchCV(lr_pipeline_poly, degrees, cv = 5, scoring='r2')

In [102]:
gscv_fitted = gscv.fit(X, y)

#gscv_fitted.cv_results_

In [103]:
#gscv_fitted.cv_results_['mean_test_score']

In [104]:
results = gscv.cv_results_

# Create a DataFrame with the degree combinations and corresponding mean test scores
df_results = pd.DataFrame({
    "degree_area": results['param_preprocessing__polynomial_area__degree'],
    "degree_rooms": results['param_preprocessing__polynomial_rooms__degree'],
    "mean_test_score": results['mean_test_score']
})

df_results
df_results.loc[df_results['mean_test_score'].idxmax()]

degree_area               3
degree_rooms              1
mean_test_score    0.557641
Name: 20, dtype: object

Q1:
The best model had an R-Squared value of about 0.56 after cross validation. This occurred with an area degree of 3, a room degree of 1, and building type in the model. These R-Squared values were calculated using cross validation with 5 folds, and we picked from 100 possible models.

Q2:
Some of the downsides of finding this many models are it takes awhile to run, and there is a factor of randomness to it. Since there are so many models, there is a good chance that one of them just happened to perform the best on this test set and might not have actually been the best model. Using this many models also includes a risk of overfitting the model. It's best to find smaller subsets to test.

Chat GPT 4o was used to debug some syntax errors and to help create a table for 100 models.